Importar datos de Cartas de OP

In [24]:
import pandas as pd
One_Piece_cards =  pd.read_csv('Cartas_One_Piece.csv')

Filtrar cartas de rarezas Secret Rare, Special y Alternate Art

In [25]:
Rarity_list = ['Secret Rare', 'Special', 'Alternate Art']
One_Piece_cards_wishlist =One_Piece_cards[One_Piece_cards['Rarity'].isin(Rarity_list)]

Credenciales para conexión con API

In [26]:
url = 'https://api.cardtrader.com/api/v2/marketplace/products'

# Tu token de autenticación
auth_token = 'TOKEN_AUTORIZACION_CARDTRADER'
# 
# Cabeceras de la solicitud
headers = {
    "Authorization": f"Bearer {auth_token}"
}

Realizar consulta a API de Cardtrader de precio de cada carta y estructura la data

In [27]:
import requests
import time

all_cards = []
cartas_precios = pd.Series()

Card_ids_wishlist = One_Piece_cards_wishlist['Card_id'].values

for bp in Card_ids_wishlist:
# Parámetros opcionales de la solicitud
    params = {
        
        'blueprint_id' : bp
    }
    response = requests.get(url, headers=headers, params=params)
    # Verificar el estado de la respuesta
    if response.status_code == 200:
        data = response.json()

        if not data[str(bp)]:
            continue  

        df = pd.json_normalize(data[str(bp)])
        card_df = df[['id' ,'blueprint_id',
                    'properties_hash.condition',
                    'properties_hash.collector_number','name_en',
                    'properties_hash.onepiece_language','expansion.code', 
                    'properties_hash.onepiece_rarity',
                    'user.id',
                    'user.username',
                    'user.country_code',
                    'quantity','price_cents']]
        
        mask_condition = (card_df[ 'properties_hash.onepiece_language'] == 'en' )& (card_df[ 'properties_hash.condition'] == 'Near Mint' )
        filter_card = card_df[mask_condition]
        filter_card.loc[:, 'price_cents'] = filter_card['price_cents'] / 100
        filter_card = filter_card.drop(['properties_hash.condition','properties_hash.onepiece_language'], axis=1)
        filter_card.columns = ['id', 'Card_id', 'Collector_code', 'Card_name','Expansion_code','Rarity', 'User_id', 'User_name','User_country','Quantity','Price']

        all_cards.append(filter_card)
    else:
        print(f"Error: {response.status_code} - {response.text}")
    time.sleep(1)

cartas_precios = pd.concat(all_cards, ignore_index=True)

Eliminar valores atipicos de precios de cartas con zscore

In [28]:
from scipy import stats

# Calcular Z-scores para la columna 'Price' por 'Card_id'
cartas_precios['z_score'] = cartas_precios.groupby('Card_id')['Price'].transform(lambda x: stats.zscore(x))

# Filtrar los valores con un Z-score mayor que 3 o menor que -3
cartas_precios = cartas_precios[cartas_precios['z_score'].abs() <= 3]
cartas_precios = cartas_precios.drop('z_score',axis=1)

Guardar datos extraidos en una base de datos MySQL.

In [29]:
import pymysql
def send_to_mysql(data):
    try:
        connection = pymysql.connect(
            host="localhost",
            user="root",
            password="miclave",
            database='One_Piece_Cards'
        )
        cursor = connection.cursor()

        # Crear tabla si no existe
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS OP_Current_Cards (
            id INT PRIMARY KEY,
            Card_id INT,
            Collector_code VARCHAR(100),
            Card_Name VARCHAR(100),
            Expansion_Code VARCHAR(100),
            Rarity VARCHAR(100),
            User_id INT,
            User_name VARCHAR(100),
            User_country VARCHAR(100),
            Quantity VARCHAR(100),
            Price DECIMAL(10,2)
        )
        """)

        # Insertar datos
        columns = ', '.join(data.columns)
        values = ', '.join(['%s'] * len(data.columns))  
        update_values = ', '.join([f"{col}=VALUES({col})" for col in data.columns if col != 'id'])
        for _, row in data.iterrows():
            cursor.execute(f"""
            INSERT INTO OP_Current_Cards ({columns}) 
            VALUES ({values}) 
            ON DUPLICATE KEY UPDATE {update_values}
            """, (tuple(row)))

        connection.commit()
        print("Datos insertados en MySQL.")
    except pymysql.MySQLError as e:
        print("Error en MySQL:", e)
    finally:
        if connection:
            cursor.close()
            connection.close()


In [30]:
send_to_mysql(cartas_precios)

Datos insertados en MySQL.


Dataset de metricas de precio(promedio, min, max) de cada carta.

In [31]:
import time

# Agregar timestamp
timestamp = time.strftime('%Y-%m-%d %H:%M:%S')

# Agrupar por Card_ID y calcular las métricas (máximo, mínimo, promedio)
df_metrics = cartas_precios.groupby('Card_id').agg(
    Collector_code = ('Collector_code','first'), 
    Card_name = ('Card_name','first'),
    Expansion_code = ('Expansion_code','first'),
    Rarity = ('Rarity','first'),
    Quantity = ('Quantity','sum'),                                 
    Max_Price=('Price', 'max'),
    Min_Price=('Price', 'min'),
    Avg_Price=('Price', 'mean')
).reset_index()

# Agregar el timestamp a las métricas
df_metrics['Timestamp'] = timestamp

Guardar en base de datos MySQL

In [32]:
def send_to_mysql(data):
    try:
        connection = pymysql.connect(
            host="localhost",
            user="root",
            password="miclave",
            database='One_Piece_Cards'
        )
        cursor = connection.cursor()

        # Crear tabla si no existe
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS OP_Current_Price_Metrics (
            Card_id INT PRIMARY KEY,
            Collector_code VARCHAR(100),
            Card_Name VARCHAR(100),
            Expansion_Code VARCHAR(100),
            Rarity VARCHAR(100),
            Quantity INT,
            Max_Price DECIMAL(10,2),
            Min_Price DECIMAL(10,2),
            Avg_Price DECIMAL(10,2),
            Timestamp DATETIME
        )
        """)

        # Insertar datos
        columns = ', '.join(data.columns)
        values = ', '.join(['%s'] * len(data.columns))  
        update_values = ', '.join([f"{col}=VALUES({col})" for col in data.columns if col != 'id'])
        for _, row in data.iterrows():
            cursor.execute(f"""
            INSERT INTO OP_Current_Price_Metrics ({columns}) 
            VALUES ({values}) 
            ON DUPLICATE KEY UPDATE {update_values}
            """, (tuple(row)))

        connection.commit()
        print("Datos insertados en MySQL.")
    except pymysql.MySQLError as e:
        print("Error en MySQL:", e)
    finally:
        if connection:
            cursor.close()
            connection.close()

send_to_mysql(df_metrics)

Datos insertados en MySQL.
